In [4]:
import os
import sys
#sys.path.append("/workspace")
import numpy as np
import tensorflow as tf
from src.data.general_processor import Utils
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, EarlyStopping
import pickle
from sklearn.preprocessing import minmax_scale
import pathlib
from sklearn.metrics import classification_report, confusion_matrix


from src.models.model_architectures.model_1DCNN import HopefullNet
from src.data.make_dataset import BCIDataset

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
root_path = pathlib.Path().resolve().parents[0]

# DATA
data_root = "C:/Users/annag/OneDrive - Danmarks Tekniske Universitet/Semester_04/Special_Course_BCI/03_code/BCI_stroke_rehab/data/raw/"

# MODEL
save_path = os.path.join(root_path, "models/1D_CNN/ours_3_pairs/leave_subjects_out/correctedBug/")

## Device settings
tf.autograph.set_verbosity(0)
physical_devices = tf.config.experimental.list_physical_devices('CPU')

#Params    
channels = [["C3","C4"],["F3","F4"],["P3","P4"]]
two_class = True

if two_class:
    input_shape = (None, 500, 2)
    loss = tf.keras.losses.binary_crossentropy
else:
    input_shape = (None, 640, 2)
    loss = tf.keras.losses.categorical_crossentropy

In [6]:
# Test subjects for testing transfer learning
test_subjects = ["dani", "ivo", "pablo", "huiyu", "manu", "fabio", "anna", "luisa", "sarah", "irene", "jan"]
measurements = []

# Get dataset
trainingset = BCIDataset(data_root, test_subjects, [], measurement_length=4)
trainingset.validate_data()
trainingset.apply_bandpass_filtering(selected_data="sample")

Subjects: ['dani', 'ivo', 'pablo', 'huiyu', 'manu', 'fabio', 'anna', 'luisa', 'sarah', 'irene', 'jan']
Total invalid samples: 63


In [8]:
for subj in test_subjects:
    print(f"Test model with subject {subj}:\n")
    
    # Get model path
    model_path = os.path.join(save_path+"test_"+subj+"/correctedBug/", "bestModel.h5")

    # Load data
    x, y_test,_ = trainingset.load_subject_data(subj, channels)

    # Reshape for scaling
    reshaped_x = x.reshape(x.shape[0], x.shape[1] * x.shape[2])
    reshaped_x_scaled = minmax_scale(reshaped_x, axis=1)
    # x_test = reshaped_x_scaled.reshape(reshaped_x_scaled.shape[0], int(reshaped_x_scaled.shape[1]/2),2).astype(np.float64)
    x_test = reshaped_x_scaled.reshape(reshaped_x_scaled.shape[0], x.shape[1], x.shape[2]).astype(np.float64)
    x_test = np.swapaxes(x_test,1,2)

    # Transform labels in int-values (0/1)
    y_test_01 = []
    for y_label in y_test:
        if y_label == 'L':
            y_test_01.append(0)
        elif y_label == 'R':
            y_test_01.append(1)
        else:
            print("Test Labels are different than L or R...")

    y_test = np.array(y_test_01)
        
    ## Test model with subject data    
    model = HopefullNet(inp_shape = (input_shape[1],input_shape[2]), two_class=two_class)
    model.build(input_shape)
    model.load_weights(model_path)

    learning_rate = 1e-4
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    
    testLoss, testAcc = model.evaluate(x_test, y_test)
    print('\nAccuracy:', testAcc)
    print('\nLoss: ', testLoss)

    yPred = model.predict(x_test)

    # convert label in string
    if two_class:
        yTestClass = []
        for label in y_test:
            if label==0:
                yTestClass.append("L")
            elif label==1:
                yTestClass.append("R")
            else:
                print("Label not found.")

        yPredClass = []
        for label in yPred:
            if label<0.5:
                yPredClass.append("L")
            elif label>=0.5:
                yPredClass.append("R")
            else:
                print("Label not found.")

        target_names = ["L", "R"]
    else:  
        yTestClass = np.argmax(y_test, axis=1)
        yPredClass = np.argmax(yPred,axis=1)
        target_names=["B", "R", "RL", "L", "F"]

    print('\n Classification report \n\n',
      classification_report(
          yTestClass,
          yPredClass,
           target_names=target_names
      )
    )

    print('\n Confusion matrix \n\n',
      confusion_matrix(
          yTestClass,
          yPredClass,
      )
    )

Test model with subject dani:

Get data from subjects: ['dani']
Predicting 2 classes.
12/12 [==============================] - 2s 39ms/step - loss: 0.7182 - accuracy: 0.4361

Accuracy: 0.43611112236976624

Loss:  0.7181910872459412
12/12 [==============================] - 1s 33ms/step

 Classification report 

               precision    recall  f1-score   support

           L       0.44      0.43      0.43       180
           R       0.44      0.44      0.44       180

    accuracy                           0.44       360
   macro avg       0.44      0.44      0.44       360
weighted avg       0.44      0.44      0.44       360


 Confusion matrix 

 [[ 78 102]
 [101  79]]
Test model with subject ivo:

Get data from subjects: ['ivo']
Predicting 2 classes.
12/12 [==============================] - 1s 28ms/step - loss: 0.6942 - accuracy: 0.4611

Accuracy: 0.4611110985279083

Loss:  0.694181501865387
12/12 [==============================] - 1s 33ms/step

 Classification report 

       